In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load CIFAR-10 dataset from the datasets library
dataset = load_dataset("cifar10")

In [ ]:
# Define a feature extractor for ViT model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

In [ ]:
# Preprocess the dataset
def preprocess_dataset(examples):
    inputs = feature_extractor(examples['image'], return_tensors='pt')
    return {**inputs, 'label': examples['label']}

train_dataset = dataset['train'].map(preprocess_dataset)
test_dataset = dataset['test'].map(preprocess_dataset)

In [ ]:

# Define a ViT model for image classification
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=10)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./vit_cifar10",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_steps=500,
    save_total_limit=3,
    push_to_hub=False,
)

In [ ]:
# Define a Trainer for training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Train the model
trainer.train()


In [ ]:

# Evaluate the model
results = trainer.evaluate()


In [ ]:

# Calculate and print accuracy
accuracy = results['eval_accuracy']
print(f"Accuracy: {accuracy:.2f}")


In [ ]:

# Save the model
model.save_pretrained("./vit_cifar10_model")


In [ ]:

# Load the model
model = ViTForImageClassification.from_pretrained("./vit_cifar10_model")


In [ ]:

# Load test data and preprocess
test_data = dataset['test']['image']
test_input = feature_extractor(test_data, return_tensors='pt')


In [ ]:

# Make predictions
with torch.no_grad():
    logits = model(**test_input).logits


In [ ]:

# Convert logits to predictions
predictions = np.argmax(logits, axis=1)


In [ ]:

# Calculate accuracy
true_labels = dataset['test']['label']
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.2f}")